# Square-root UKF

## 1. Square-root Covariance update with QR-Decomposition

Knowing that:

$$ \tag{1}
A+B = \begin{bmatrix} \sqrt{A}^T & \sqrt{B}^T \end{bmatrix} \begin{bmatrix} \sqrt{A} \\ \sqrt{B} \end{bmatrix}
$$

$$ \tag{2}
Q, R = qr \left( \begin{bmatrix} \sqrt{A} \\ \sqrt{B} \end{bmatrix} \right)
$$

where $Q$ is an orthogonal matrix, and $R$ is upper triangular.

Substitue the $QR$ decomposition $(2)$ into $(1)$:

$$ \tag{3}
A + B = R^T Q^T Q R
$$

From the properties of the orthogonal matrix is that its inverse is equal to its transpose **(Theorem 4.1, 4.2)** in [1].

$$ \tag{4}
Q^T = Q^{-1}
$$

Note: one can relate that this is a similar property as the rotation matrix.

Because of this property the dot product of the orthogonal matrix with its transpose is equal to identiy matrix.

$$ \tag{5}
Q.Q^T = I
$$

substituding $(5)$ into $(3)$ yielf;

$$ \tag{6}
A + B = R^T R
$$

then;

$$ \tag{7}
\sqrt{A + B} = R^T
$$

and we can prove this by example, given:

$$ \tag{8}
A = \begin{bmatrix} 100 & 2 \\ 2 & 9 \end{bmatrix}
$$

and,

$$ \tag{9}
B = \begin{bmatrix} 9 & 3 \\ 3 & 4 \end{bmatrix}
$$

In [2]:
import numpy as np
from scipy import linalg

In [3]:
A = np.array([[100, 2], [2, 9]])
B = np.array([[9, 3], [3, 4]])

print(f' A + B = \n {A + B}')

 A + B = 
 [[109   5]
 [  5  13]]


In [4]:
sqrtA = np.linalg.cholesky(A).T # transpose to get the upper triangular matrix as expected by QR decomposition
sqrtB = np.linalg.cholesky(B).T # transpose to get the upper triangular matrix as expected by QR decomposition

print(f'sqrt(A) = \n{sqrtA}')
print(f'sqrt(B) = \n{sqrtB}')

sqrt(A) = 
[[10.          0.2       ]
 [ 0.          2.99332591]]
sqrt(B) = 
[[3.         1.        ]
 [0.         1.73205081]]


In [5]:
C = np.concatenate((sqrtA, sqrtB), axis=0) # building the compound (joint) matrix
print(C)
Q, R = np.linalg.qr(C)
print(f'Q=\n{Q}')
print(f'R=\n{R}')

[[10.          0.2       ]
 [ 0.          2.99332591]
 [ 3.          1.        ]
 [ 0.          1.73205081]]
Q=
[[-0.95782629  0.07239628]
 [-0.         -0.83762115]
 [-0.28734789 -0.24132093]
 [-0.         -0.48467906]]
R=
[[-10.44030651  -0.47891314]
 [  0.          -3.57360353]]


In [6]:
A_plus_B = R.T @ R
print(f'R.T @ R = \n{A_plus_B}\n')

R.T @ R = 
[[109.   5.]
 [  5.  13.]]



## 2. Rank 1 Cholesky Update/Downdate

$$
A = LL^T
$$

$$
A'= A + \beta \nu \nu^T
$$

$A$ is positive definite matrix as well as $A'$.

$$
A' = L' L'^T
$$

$$
LL^T = L'L'^T + \beta \nu \nu^T
$$

$$
L' = \sqrt{LL^T + \beta \nu \nu^T}
$$

In [7]:
def cholupdate(L, W, beta):
    r = np.shape(W)[1]
    m = np.shape(L)[0]
       
    for i in range(r):
        L_out = np.copy(L)
        b = 1.0
        
        for j in range(m):
            Ljj_pow2 = L[j, j]**2
            wji_pow2 = W[j, i]**2
            
            L_out[j, j] = np.sqrt(Ljj_pow2 + (beta / b) * wji_pow2)
            upsilon = (Ljj_pow2 * b) + (beta * wji_pow2)
            
            for k in range(j+1, m):
                W[k, i] -= (W[j, i] / L[j,j]) * L[k,j]
                L_out[k, j] = ((L_out[j, j] / L[j, j]) * L[k,j]) + (L_out[j, j] * beta * W[j, i] * W[k, i] / upsilon)
            
            b += beta * (wji_pow2 / Ljj_pow2)
        
        L = np.copy(L_out)
    
    return L_out

In [8]:
beta = 1

A = np.array([[100, 2], [2, 9]])

v = np.array([[3], [2]])

A2 = A + beta * (v @ v.T)

print(f'A + beta * (v @ v.T) = \n{A2}\n')

A + beta * (v @ v.T) = 
[[109   8]
 [  8  13]]



In [9]:
L = np.linalg.cholesky(A)

L2 = cholupdate(L, v, 1.)

print(f'L2 @ L2.T = \n{L2 @ L2.T}\n')

L2 @ L2.T = 
[[109.       5.18  ]
 [  5.18    10.1236]]



In [10]:
beta = 1
A = np.array([[100., 2.], [2., 9.]])
W = np.array([[3., 1.], [1., 2.]])

A1 = A + beta * (W @ W.T)
print(f'A + beta * (W @ W.T) = \n{A1}\n')

L1 = np.linalg.cholesky(A)
    
L2 = cholupdate(L1, W, 1.0)
A2 = L2 @ L2.T
print(f'L2 @ L2.T = \n{A2}\n')

A + beta * (W @ W.T) = 
[[110.   7.]
 [  7.  14.]]

L2 @ L2.T = 
[[110.   7.]
 [  7.  14.]]



## 3. Backward Substitution

In [11]:
def backsubs(A, B):
    # x_ik = (b_ik - Sum_aij_xjk) / a_ii
    
    N = np.shape(A)[0]
    
    X = np.zeros((B.shape[0], B.shape[1]))
    
    for k in range(B.shape[1]):
        for i in range(N-1, -1, -1):
            sum_aij_xj = B[i, k]

            for j in range(N-1, i, -1):
                sum_aij_xj -= A[i, j] * X[j, k]

            X[i, k] = sum_aij_xj / A[i, i]
    
    return X

In [12]:
A = np.array([[1., -2., 1.],
              [0., 1., 6.],
              [0., 0., 1.]])

b = np.array([[4.0], [-1.0], [2.0]])

x1 = backsubs(A, b)
print(x1)

[[-24.]
 [-13.]
 [  2.]]


## 4. Forward Substitution

In [13]:
def forwardsubs(A, B):
    # x_ik = (b_ik - Sum_aij_xjk) / a_ii
    
    N = np.shape(A)[0]
    X = np.zeros((B.shape[0], B.shape[1]))
    
    for k in range(B.shape[1]):
        for i in range(N):
            sum_aij_xj = B[i, k]
            
            for j in range(i):
                sum_aij_xj -= A[i, j] * X[j, k]
                
            X[i, k] = sum_aij_xj / A[i, i]
    
    return X

In [14]:
A = np.array([[1., -2., 1.],
              [0., 1., 6.],
              [0., 0., 1.]])

A = A.T

b = np.array([[4.0], [-1.0], [2.0]])

x2 = forwardsubs(A, b)

print(x2)

[[  4.]
 [  7.]
 [-44.]]


## 5. Square-root UKF

In [39]:
class SquareRootUKF(object):
    def __init__(self, x, P, Q, R):              
        self.dim_x = np.shape(x)[0]
        self.n_sigmas = (2 * self.dim_x) + 1
        
        self.kappa = 3 - self.dim_x
        
        self.sigma_scale = np.sqrt(self.dim_x + self.kappa)
        
        self.W0 = self.kappa / (self.dim_x + self.kappa)
        self.Wi = 0.5 / (self.dim_x + self.kappa)
        
        self.x = x
        
        # lower triangular matrices
        self.sqrt_P = np.linalg.cholesky(P)
        self.sqrt_Q = np.linalg.cholesky(Q)
        self.sqrt_R = np.linalg.cholesky(R)
        
        print(f'R = \n{R}\n')
    
    def predict(self, f):
        # generate sigma points
        sigmas_X = self.sigma_points(self.x, self.sqrt_P)
        
        # propagate sigma points through the nonlinear function
        for i in range(self.n_sigmas):
            sigmas_X[:, i] = f(sigmas_X[:, i])
        
        # calculate weighted mean
        x_minus = self.W0 * sigmas_X[:, 0]
        for i in range(1, self.n_sigmas):
            x_minus += self.Wi * sigmas_X[:, i]
        
        # build compound matrix for square-root covariance update
        # sigmas_X[:, 0] is not added because W0 could be zero which will lead
        # to undefined outcome from sqrt(W0).
        C = (sigmas_X[:, 1:].T - x_minus) * np.sqrt(self.Wi)
        C = np.concatenate((C, self.sqrt_Q.T), axis=0)
        
        # calculate square-root covariance S using QR decomposition of compound matrix C
        # including the process noise covariance
        Q , S_minus = np.linalg.qr(C)
        print(f'Q = \n{Q}\n')
        print(f'R = \n{S_minus}\n')
        
        # Rank-1 cholesky update
        x_dev = sigmas_X[:, 0] - x_minus
        x_dev = np.reshape(x_dev, [-1, 1])
        print(f'x_dev = \n{x_dev}\n')
        S_minus = cholupdate(S_minus.T, x_dev, self.W0)
        
        # overwrite member x and S
        self.x = x_minus
        self.sqrt_P = S_minus
        
        print(f'S^- = \n{S_minus}\n')
    
    
    def correct(self, h, z):
        # generate sigma points X
        sigmas_X = self.sigma_points(self.x, self.sqrt_P)
        
        # propagate sigma points X through the nonlinear function
        # to get output sigma points Y
        dim_z = np.shape(z)[0]
        sigmas_Y = np.zeros((dim_z, self.n_sigmas))
        for i in range(self.n_sigmas):
            sigmas_Y[:, i] = h(sigmas_X[:, i])
        
        print(f'Ys = \n{sigmas_Y}\n')
        
        # calculate weighted mean y
        y_bar = self.W0 * sigmas_Y[:, 0]
        for i in range(1, self.n_sigmas):
            y_bar += self.Wi * sigmas_Y[:, i]
    
        print(f'y = \n{y_bar}\n')
        
        # build compound matrix for square-root covariance update      
        C = (sigmas_Y[:, 1:].T - y_bar) * np.sqrt(self.Wi)       
        C = np.concatenate((C, self.sqrt_R.T), axis=0)
        
        print(f'sqrt_R.T = \n{self.sqrt_R.T}\n')
        
        # calculate square-root covariance S using QR decomposition of compound matrix C
        # including the process noise covariance
        _ , S_y = np.linalg.qr(C)
        
        # Rank-1 cholesky update
        y_dev = sigmas_Y[:, 0] - y_bar
        y_dev = np.reshape(y_dev, [-1, 1])
        S_y = cholupdate(S_y.T, y_dev, self.W0)
        print(f'Sy = \n{S_y}\n')
           
        # calculate cross-correlation
        Pxy = self.calculate_cross_correlation(self.x, sigmas_X, y_bar, sigmas_Y)
        print(f'Pxy = \n{Pxy}\n')
        
        # Kalman gain calculation with two nested least-squares
        # Step1: Forward-substitution -> K = Sy \ Pxy  (since S_y is lower-triangular)
        # Step2: Backward-substitution -> K = Sy.T \ K (since S_y.T is upper-triangular)
        K = forwardsubs(S_y, Pxy)
        K = backsubs(S_y.T, K)
        print(f'K = \n{K}\n')
        
        # update state vector x
        self.x += K @ (z - y_bar)
        
        # update state square-root covariance Sk
        # S_y must be upper triangular matrix at this place
        U = K @ S_y
        
        #self.sqrt_P = r_rank_cholupdate_v2(self.sqrt_P, U, -1.0)
        self.sqrt_P = cholupdate(self.sqrt_P, U, -1.0)
    
    
    def sigma_points(self, x, sqrt_P):
        
        '''
        generating sigma points matrix x_sigma given mean 'x' and square-root covariance 'S'
        '''
               
        sigmas_X = np.zeros((self.dim_x, self.n_sigmas))       
        sigmas_X[:, 0] = x

        for i in range(self.dim_x):
            idx_1 = i + 1
            idx_2 = i + self.dim_x + 1
            
            sigmas_X[:, idx_1] = x + (self.sigma_scale * sqrt_P[:, i])
            sigmas_X[:, idx_2] = x - (self.sigma_scale * sqrt_P[:, i])
            
        return sigmas_X
    
    
    def calculate_cross_correlation(self, x, x_sigmas, y, y_sigmas):
        xdim = np.shape(x)[0]
        ydim = np.shape(y)[0]
        
        n_sigmas = np.shape(x_sigmas)[1]
    
        dx = (x_sigmas[:, 0] - x).reshape([-1, 1])
        dy = (y_sigmas[:, 0] - y).reshape([-1, 1])
        Pxy = self.W0 * (dx @ dy.T)
        for i in range(1, n_sigmas):
            dx = (x_sigmas[:, i] - x).reshape([-1, 1])
            dy = (y_sigmas[:, i] - y).reshape([-1, 1])
            Pxy += self.Wi * (dx @ dy.T)
    
        return Pxy

In [29]:
def linear_func(x):
    return x

In [30]:
x = np.array([0., 0.])
P = np.array([[2.0, -0.8], [-0.8, 2.0]])

Q = np.array([[3, 0], [0, 3]])
R = np.array([[1, 0], [0, 1]])

sr_ukf = SquareRootUKF(x, P, Q, R)

sr_ukf.predict(linear_func)
print(f'P+Q = \n{P+Q}')

Q = 
[[-0.4472136   0.10873206]
 [-0.         -0.4152274 ]
 [ 0.4472136  -0.10873206]
 [-0.          0.4152274 ]
 [-0.77459667 -0.12555296]
 [-0.         -0.78470603]]

R = 
[[-2.23606798  0.35777088]
 [ 0.         -2.20726075]]

x_dev = 
[[0.]
 [0.]]

S^- = 
[[ 2.23606798  0.        ]
 [-0.35777088  2.20726075]]

P+Q = 
[[ 5.  -0.8]
 [-0.8  5. ]]


In [31]:
x0 = np.array([1.0, 2.0])
P0 = np.array([[1.0, 0.5], [0.5, 1.0]])
Q = np.array([[0.5, 0.0], [0.0, 0.5]])

z = np.array([1.2, 1.8])
R = np.array([[0.3, 0.0], [0.0, 0.3]])

In [32]:
def KF_predict(F, x, P, Q):
    x = (F @ x)
    P = F @ P @ F.T + Q
    return x, P

def KF_correct(H, z, R, x, P):
    Pxz = P @ H.T   
    S = H @ P @ H.T + R
   
    K = Pxz @ np.linalg.pinv(S)
    
    x = x + K @ (z - H @ x)
    I = np.eye(P.shape[0])
    P = (I - K @ H) @ P
    return x, P

In [33]:
F = np.array([[1.0, 0.0], [0.0, 1.0]])
H = np.array([[1.0, 0.0], [0.0, 1.0]])

x1, P1 = KF_predict(F, x0, P0, Q)

print(f'x1 = \n{x1.round(5)}\n')
print(f'P1 = \n{P1.round(5)}\n')

x2, P2 = KF_correct(H, x1, P1, z, R)

print(f'x2 = \n{x2.round(5)}\n')
print(f'P2 = \n{P2.round(5)}\n')

x1 = 
[1. 2.]

P1 = 
[[1.5 0.5]
 [0.5 1.5]]

x2 = 
[1.15385 1.84615]

P2 = 
[[0.24582 0.01505]
 [0.01505 0.24582]]



In [40]:
sr_ukf = SquareRootUKF(x0, P0, Q, R)

sr_ukf.predict(linear_func)

x1 = sr_ukf.x
P1 = sr_ukf.sqrt_P @ sr_ukf.sqrt_P.T

print(f'x1 = \n{x1.round(5)}\n')
print(f'P1 = \n{P1.round(5)}\n')

sr_ukf.correct(linear_func, z)

x2 = sr_ukf.x
P2 = sr_ukf.sqrt_P @ sr_ukf.sqrt_P.T

print(f'x2 = \n{x2.round(5)}\n')
print(f'P2 = \n{P2.round(5)}\n')
print(f'S2 = \n{sr_ukf.sqrt_P.round(5)}\n')

R = 
[[0.3 0. ]
 [0.  0.3]]

Q = 
[[-5.77350269e-01 -1.02062073e-01]
 [-3.70074342e-17 -5.30330086e-01]
 [ 5.77350269e-01  1.02062073e-01]
 [-3.70074342e-17  5.30330086e-01]
 [-5.77350269e-01  2.04124145e-01]
 [-0.00000000e+00 -6.12372436e-01]]

R = 
[[-1.22474487 -0.40824829]
 [ 0.         -1.15470054]]

x_dev = 
[[1.11022302e-16]
 [0.00000000e+00]]

S^- = 
[[1.22474487 0.        ]
 [0.40824829 1.15470054]]

x1 = 
[1. 2.]

P1 = 
[[1.5 0.5]
 [0.5 1.5]]

Ys = 
[[ 1.00000000e+00  3.12132034e+00  1.00000000e+00 -1.12132034e+00
   1.00000000e+00]
 [ 2.00000000e+00  2.70710678e+00  4.00000000e+00  1.29289322e+00
   2.22044605e-16]]

y = 
[1. 2.]

sqrt_R.T = 
[[0.54772256 0.        ]
 [0.         0.54772256]]

Sy = 
[[1.34164079 0.        ]
 [0.372678   1.288841  ]]

Pxy = 
[[1.5 0.5]
 [0.5 1.5]]

K = 
[[0.81939799 0.05016722]
 [0.05016722 0.81939799]]

x2 = 
[1.15385 1.84615]

P2 = 
[[0.24582 0.01505]
 [0.01505 0.24582]]

S2 = 
[[0.4958  0.     ]
 [0.03036 0.49487]]



In [21]:
class UKF(object):
    def __init__(self, dim_x, dim_z, Q, R, kappa=0.0):
        
        '''
        UKF class constructor
        inputs:
            dim_x : state vector x dimension
            dim_z : measurement vector z dimension
        
        - step 1: setting dimensions
        - step 2: setting number of sigma points to be generated
        - step 3: setting scaling parameters
        - step 4: calculate scaling coefficient for selecting sigma points
        - step 5: calculate weights
        '''
                
        # setting dimensions
        self.dim_x = dim_x         # state dimension
        self.dim_z = dim_z         # measurement dimension
        self.dim_v = np.shape(Q)[0]
        self.dim_n = np.shape(R)[0]
        self.dim_a = self.dim_x + self.dim_v + self.dim_n # assuming noise dimension is same as x dimension
        
        # setting number of sigma points to be generated
        self.n_sigma = (2 * self.dim_a) + 1
        
        # setting scaling parameters
        self.kappa = 3 - self.dim_a #kappa
        self.alpha = 0.001
        self.beta = 2.0

        alpha_2 = self.alpha**2
        self.lambda_ = alpha_2 * (self.dim_a + self.kappa) - self.dim_a
        
        # setting scale coefficient for selecting sigma points
        # self.sigma_scale = np.sqrt(self.dim_a + self.lambda_)
        self.sigma_scale = np.sqrt(self.dim_a + self.kappa)
        
        # calculate unscented weights
        # self.W0m = self.W0c = self.lambda_ / (self.dim_a + self.lambda_)
        # self.W0c = self.W0c + (1.0 - alpha_2 + self.beta)
        # self.Wi = 0.5 / (self.dim_a + self.lambda_)
        
        self.W0 = self.kappa / (self.dim_a + self.kappa)
        self.Wi = 0.5 / (self.dim_a + self.kappa)
        
        # initializing augmented state x_a and augmented covariance P_a
        self.x_a = np.zeros((self.dim_a, ))
        self.P_a = np.zeros((self.dim_a, self.dim_a))
        
        self.idx1, self.idx2 = self.dim_x, self.dim_x + self.dim_v
        
        self.P_a[self.idx1:self.idx2, self.idx1:self.idx2] = Q
        self.P_a[self.idx2:, self.idx2:] = R
        
        print(f'P_a = \n{self.P_a}\n')
            
    def predict(self, f, x, P):       
        self.x_a[:self.dim_x] = x
        self.P_a[:self.dim_x, :self.dim_x] = P
        
        xa_sigmas = self.sigma_points(self.x_a, self.P_a)
        
        xx_sigmas = xa_sigmas[:self.dim_x, :]
        xv_sigmas = xa_sigmas[self.idx1:self.idx2, :]
        
        y_sigmas = np.zeros((self.dim_x, self.n_sigma))              
        for i in range(self.n_sigma):
            y_sigmas[:, i] = f(xx_sigmas[:, i], xv_sigmas[:, i])
        
        y, Pyy = self.calculate_mean_and_covariance(y_sigmas)
        
        self.x_a[:self.dim_x] = y
        self.P_a[:self.dim_x, :self.dim_x] = Pyy
               
        return y, Pyy, xx_sigmas
        
    def correct(self, h, x, P, z):
        self.x_a[:self.dim_x] = x
        self.P_a[:self.dim_x, :self.dim_x] = P
        
        xa_sigmas = self.sigma_points(self.x_a, self.P_a)
        
        xx_sigmas = xa_sigmas[:self.dim_x, :]
        xn_sigmas = xa_sigmas[self.idx2:, :]
        
        y_sigmas = np.zeros((self.dim_z, self.n_sigma))
        for i in range(self.n_sigma):
            y_sigmas[:, i] = h(xx_sigmas[:, i], xn_sigmas[:, i])
            
        y, Pyy = self.calculate_mean_and_covariance(y_sigmas)
                
        Pxy = self.calculate_cross_correlation(x, xx_sigmas, y, y_sigmas)
        print(f'Pxy = \n {Pxy}')

        K = Pxy @ np.linalg.pinv(Pyy)
        print(f'K = \n {K}')
        
        x = x + (K @ (z - y))
        
        print(f'S = \n{np.linalg.cholesky(P)}')
        P = P - (K @ Pyy @ K.T)
        
        return x, P, xx_sigmas
        
    
    def sigma_points(self, x, P):
        
        '''
        generating sigma points matrix x_sigma given mean 'x' and covariance 'P'
        '''
        
        nx = np.shape(x)[0]
        
        x_sigma = np.zeros((nx, self.n_sigma))       
        x_sigma[:, 0] = x
        
        S = np.linalg.cholesky(P)
        
        for i in range(nx):
            x_sigma[:, i + 1]      = x + (self.sigma_scale * S[:, i])
            x_sigma[:, i + nx + 1] = x - (self.sigma_scale * S[:, i])
            
        return x_sigma
    
    
    def calculate_mean_and_covariance(self, y_sigmas):
        ydim = np.shape(y_sigmas)[0]
        
        # mean calculation
        y = self.W0 * y_sigmas[:, 0]
        for i in range(1, self.n_sigma):
            y += self.Wi * y_sigmas[:, i]
            
        # covariance calculation
        d = (y_sigmas[:, 0] - y).reshape([-1, 1])
        Pyy = self.W0 * (d @ d.T)
        for i in range(1, self.n_sigma):
            d = (y_sigmas[:, i] - y).reshape([-1, 1])
            Pyy += self.Wi * (d @ d.T)
    
        return y, Pyy
    
    def calculate_cross_correlation(self, x, x_sigmas, y, y_sigmas):
        xdim = np.shape(x)[0]
        ydim = np.shape(y)[0]
        
        n_sigmas = np.shape(x_sigmas)[1]
    
        dx = (x_sigmas[:, 0] - x).reshape([-1, 1])
        dy = (y_sigmas[:, 0] - y).reshape([-1, 1])
        Pxy = self.W0 * (dx @ dy.T)
        for i in range(1, n_sigmas):
            dx = (x_sigmas[:, i] - x).reshape([-1, 1])
            dy = (y_sigmas[:, i] - y).reshape([-1, 1])
            Pxy += self.Wi * (dx @ dy.T)
    
        return Pxy

In [22]:
def f(x, v):
    return (x + v)

def h(x, n):
    return (x + n)

nx = np.shape(x0)[0]
nz = np.shape(z)[0]
nv = np.shape(x0)[0]
nn = np.shape(z)[0]

ukf = UKF(dim_x=nx, dim_z=nz, Q=Q, R=R, kappa=(3 - nx))

x1, P1, _ = ukf.predict(f, x0, P0)

print(f'x = \n{x1.round(5)}\n')
print(f'P = \n{P1.round(5)}\n')

x2, P2, _ = ukf.correct(h, x1, P1, z)

print(f'x = \n{x2.round(5)}\n')
print(f'P = \n{P2.round(5)}\n')

print(f'S = \n{np.linalg.cholesky(P2)}')

# K = 
#  [[0.81939799 0.05016722]
#  [0.05016722 0.81939799]]

P_a = 
[[0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.5 0.  0. ]
 [0.  0.  0.  0.  0.3 0. ]
 [0.  0.  0.  0.  0.  0.3]]

x = 
[1. 2.]

P = 
[[1.5 0.5]
 [0.5 1.5]]

Pxy = 
 [[1.5 0.5]
 [0.5 1.5]]
K = 
 [[0.81939799 0.05016722]
 [0.05016722 0.81939799]]
S = 
[[1.22474487 0.        ]
 [0.40824829 1.15470054]]
x = 
[1.15385 1.84615]

P = 
[[0.24582 0.01505]
 [0.01505 0.24582]]

S = 
[[0.49580177 0.        ]
 [0.03035521 0.49487166]]


# References

[1] [The Orthogonal matrix and its applications](http://libres.uncg.edu/ir/uncg/f/shugart_sue_1953.pdf)

[2] [R. Van der Merwe and E. A. Wan, "The square-root unscented Kalman filter for state and parameter-estimation," 2001 IEEE International Conference on Acoustics, Speech, and Signal Processing. Proceedings (Cat. No.01CH37221), 2001, pp. 3461-3464 vol.6, doi: 10.1109/ICASSP.2001.940586.](https://www.researchgate.net/publication/3908304_The_Square-Root_Unscented_Kalman_Filter_for_State_and_Parameter-Estimation)